# Wrangling with Pandas


### The Problem

> 80% of the effort in data analysis is spent cleaning data. [Hadley Wickham](http://vita.had.co.nz/papers/tidy-data.pdf)

- Different data formats - often not good for analytics
- [Tidy data](http://vita.had.co.nz/papers/tidy-data.pdf) by Hadley Wickham- 
- Raw data is often in the wrong format
- Storage may be best in a different format

## Outline

**Simple example**

- Reshaping: `pd.pivot` and `pd.melt`
- Many basic operations: add, remove, indexing
- Missing values

**Movies**

- Joining
- Groupby
- Sorting


### Simple example

Based on [Data Wrangling Kung Fu with Pandas](vimeo.com/63295598) by Wes McKinney

In [ ]:
import os
import pandas as  pd
import numpy as np

**Setup only**: Write some data for reading

***Note:*** This Jupyter Notebook assumes that you have a subfolder or subdirectory called 'data' available. Be sure to create one before starting. 

In [ ]:
dates = ['2014-02-16', '2014-02-17', '2014-02-18', '2014-02-19']
algs = ['Model-A','Model-B','Model-C']

filename = os.path.join('data','example.csv')

with open(filename,'w') as outfile:
    outfile.write('date,type,value\n')
    for d in dates:
        for a in algs:
            v = np.random.randint(10, size=1)[0]
            tmp = '{0},{1},{2}\n'.format(d,a,v)
            outfile.write(tmp)

In [ ]:
with open(filename, 'r') as infile:
    print(infile.read())

In [ ]:
filename

### Creating a `DataFrame`

In [ ]:
df = pd.read_csv(filename)
print(df)

**Why store it this way?**

- Different type
- Different metric

### Reshape with `pivot`

- Question: What is the average value for each date?
- How many observations do I have for each model?

In [ ]:
results = df.pivot('date', 'type', 'value') #row, column, values (optional)
print(results)

In [ ]:
results.columns

In [ ]:
results.index

### Columns access

In [ ]:
results['Model-A']

In [ ]:
results['Model-A'].values

### Row access

In [ ]:
results.iloc[0]

### Range access

In [ ]:
print(results.iloc[2:4,1:])

### Summarize rows and columns

Question: What is the average value for each date?

In [ ]:
results.mean(axis=1)

How many observations do I have for each model?

In [ ]:
results.count(axis=0)

### Add some data with `pd.concat`

In [ ]:
df = pd.read_csv(filename)
tmp = {'date': ['2014-02-16','2014-02-18'],
       'type': ['Model-D', 'Model-D'],
       'value': [11, 7]}

pd.DataFrame(tmp)

In [ ]:
df = pd.concat([df,pd.DataFrame(tmp)], ignore_index=True)
df.shape

### Delete a row

In [ ]:
df.drop(2, axis=0).head()

In [ ]:
df.drop('type', axis=1).head()

### Let's `reshape` again...

In [ ]:
results = df.pivot('date','type', 'value')
print(results)

In [ ]:
results.mean(axis=1)

In [ ]:
results.count(axis=0)

In [ ]:
results.count(axis=1)

### Missing vales: `isnull()` and `fillna()`

In [ ]:
print(results.isnull())

In [ ]:
print(results.fillna(0))

In [ ]:
print(results)

In [ ]:
tmp = results.copy()

In [ ]:
tmp.fillna(0, inplace=True)
print(tmp)

### `reset_index`

In [ ]:
tmp.reset_index(inplace=True)
tmp.columns

In [ ]:
print(tmp)

### Reshape with `melt`

In [ ]:
results = df.pivot('date','type', 'value')
print(results)

In [ ]:
results.reset_index(inplace=True)
print(results)

In [ ]:
back = pd.melt(results, id_vars=['date'])
print(back)

### `dropna()`

In [ ]:
back.dropna(axis=0)

In [ ]:
back.dropna(axis=1).head()

### Write to file `to_csv`

In [ ]:
back.dropna(axis=0, inplace=True)

In [ ]:
back.to_csv('data/back.csv', index=False)

In [ ]:
print(open('data/back.csv').read())

## Movies

This is adapted from [Python for Data Analysis](http://www.amazon.com/Python-Data-Analysis-Wrangling-IPython/dp/1449319793) by Wes McKinney

- Joining
- Groupby
- Sorting

Ultimate question: What's the best **date-night movie**?

- Highly rated movie
- That appeals to both `M` and `F`

### MovieLens Data

[GroupLens Research](http://www.grouplens.org/) provides collections of movie ratings data collected from users of the recommendation service MovieLens in the late 1990s and early 2000s. The data provide movie ratings, movie metadata (genres and year), and demographic data about the users (age, zip code, gender, and occupation).

This data set contains 1,000,209 anonymous ratings of approximately 3,900 movies made by 6,040 users who joined MovieLens in 2000. It’s spread across 3 tables: ratings, user information, and movie information. 

#### File descriptions

All ratings are contained in the file "ratings.dat" and are in the
following format:

UserID::MovieID::Rating::Timestamp

- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings

User information is in the file "users.dat" and is in the following
format:

UserID::Gender::Age::Occupation::Zip-code

All demographic information is provided voluntarily by the users and is
not checked for accuracy.  Only users who have provided some demographic
information are included in this data set.

- Gender is denoted by a "M" for male and "F" for female

Movie information is in the file "movies.dat" and is in the following
format:

MovieID::Title::Genres

Titles are identical to titles provided by the IMDB (including
year of release)
Genres are pipe-separated and are selected from the following genres:

- Action
- Adventure
- Animation
- Children's
- Comedy
- Crime
- Documentary
- Drama
- Fantasy
- Film-Noir
- Horror
- Musical
- Mystery
- Romance
- Sci-Fi
- Thriller
- War
- Western

### Read the data into DataFrames

In [ ]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip'] 
users = pd.read_table('users.dat', sep='::', header=None, names=unames, engine='python')

rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ratings.dat', sep='::', header=None, names=rnames, engine='python')

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('movies.dat', sep='::', header=None, names=mnames, engine='python')

In [ ]:
users.head()

In [ ]:
ratings.head()

In [ ]:
movies.head()

Hmmmm ... `genres` has multiple entries in an ugly format. We'll deal with that later.

In [ ]:
ratings.info()

### Join the tables with `pd.merge`

Analyzing the data spread across three tables is not easy. For example, suppose you wanted to compute mean ratings for a particular movie by sex and age. This is much easier to do with all of the data merged together into a single table. Using pandas’s `merge` function, we first merge `ratings` with `users` then merge that result with the `movies` data. pandas infers which columns to use as the merge (or join) keys based on overlapping names:

In [ ]:
data = pd.merge(pd.merge(ratings, users), movies)

In [ ]:
data.info()

In [ ]:
data.head()

### What are the highest rated movies?

In [ ]:
data.sort_values(by='rating', ascending=False).head()

In [ ]:
data[['title','rating']].sort_values(by='rating', ascending=False).head()

In [ ]:
grp = data[['title','rating']].groupby('title')

In [ ]:
type(grp)

In [ ]:
mean_rating = grp.mean()

In [ ]:
mean_rating.head()

### What is the highest rated movie?

Sort by `rating`

In [ ]:
mean_rating.sort_values('rating', ascending=False).head(10)

Apply more than one function to the group with the `agg()` method.

In [ ]:
mean_rating = grp['rating'].agg(['mean','count'])

In [ ]:
mean_rating.sort_values(['mean','count'], ascending=False).head(10)

### Threshold on the number of ratings

In [ ]:
mask = mean_rating['count'] > 1000
type(mask)

In [ ]:
sum(mask)

In [ ]:
mask.head()

In [ ]:
mean_rating.loc[mask].head()

In [ ]:
mean_rating.loc[mask]['count'].min()

In [ ]:
mean_rating.loc[mask]['count'].max()

### Highest rated movie with at least 1000 votes?

In [ ]:
mean_rating.loc[mask].sort_values('mean', ascending=False).head(10)

### What about gender?

We can use a pivot table here.

In [ ]:
mean_ratings = pd.pivot_table(data, 'rating', index='title', columns='gender', aggfunc='mean')
mean_ratings.head()

Only those that have at least 1000 votes:

In [ ]:
mean_ratings = mean_ratings.loc[mask]

In [ ]:
mean_ratings.head()

### Favorites for M and for F

In [ ]:
mean_ratings.sort_values('M', ascending=False).head(10)

In [ ]:
mean_ratings.sort_values('F', ascending=False).head(10)

### Which movies differ the most and the least between M and F?

In [ ]:
mean_ratings['diff'] = abs(mean_ratings['M'] - mean_ratings['F'])

In [ ]:
mean_ratings.sort_values('diff', ascending=False).head(10)

In [ ]:
mean_ratings.sort_values('diff', ascending=True).head(10)

### Date Night pick?

What are the highest rated movies where the difference is minimal?

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
mean_ratings['diff'].hist(alpha=0.75);

In [ ]:
mean_ratings.hist(alpha=0.75);

How about diff < 0.05 and rating > 4.25?

In [ ]:
diff_mask = mean_ratings['diff'] < 0.05 
m_mask = mean_ratings['M'] > 4.25
f_mask = mean_ratings['F'] > 4.25

In [ ]:
mask = diff_mask & m_mask & f_mask

In [ ]:
date_night = mean_ratings[mask]

In [ ]:
date_night.info()

In [ ]:
date_night

In [ ]:
#ignore the warning
date_night['mean'] = date_night['M'] + date_night['F']

In [ ]:
date_night.sort_values('mean', ascending=False)